# NOTE
Ignore all dependency parse tree drawn in this file. The tool used to create those trees (spaCy) is not compatible with Stanza, so the drawn trees are not correct.

In [ ]:
import stanza
import nltk
import spacy
from spacy import displacy

In [ ]:
nlp_constituency = stanza.Pipeline(lang='en', processors='tokenize, pos, constituency')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: constituency
INFO:stanza:Done loading processors!


In [ ]:
nlp_dependency = stanza.Pipeline(lang='en', processors='tokenize, mwt, pos, lemma, depparse')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
def draw_constituency_tree(const_parsed_str):
    t = nltk.Tree.fromstring(const_parsed_str)
    t.pretty_print()

In [ ]:
def draw_dependency_tree(sent, doc_dep, dist):
  spacy_doc_list = []
  sentence = doc_dep.sentences[0].words
  for word in sentence:
    extract = (word.text, word.head, word.deprel)
    spacy_doc_list.append(extract)

  nlp = spacy.load('en_core_web_sm')
  words = [word for word, head, deprel in spacy_doc_list]
  spacy_doc = spacy.tokens.Doc(nlp.vocab, words=words)

  for i, (word, head, deprel) in enumerate(spacy_doc_list):
    token = spacy_doc[i]
    token.head = spacy_doc[head-1]
    token.dep_ = deprel

  displacy.render(docs=spacy_doc, style="dep", jupyter=True, options={"distance":dist})


In [ ]:
def print_result(sentence, dist=100):
  doc_const = nlp_constituency(sentence)
  doc_dep = nlp_dependency(sentence)
  print("Sentence: ", sentence)
  print("Constituency parse tree: ")
  doc_const_tree = str(doc_const.sentences[0].constituency)
  print(doc_const_tree)
  draw_constituency_tree(doc_const_tree)
  print("Dependency parse tree: ")
  print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc_dep.sentences for word in sent.words], sep='\n')
  draw_dependency_tree(sentence, doc_dep, dist)

# A

In [ ]:
sent_1 = "I eat spaghetti with chopsticks"

print_result(sent_1)

Sentence:  I eat spaghetti with chopsticks
Constituency parse tree: 
(ROOT (S (NP (PRP I)) (VP (VBP eat) (NP (NN spaghetti)) (PP (IN with) (NP (NNS chopsticks))))))
    ROOT                              
     |                                 
     S                                
  ___|_______                          
 |           VP                       
 |    _______|___________              
 |   |       |           PP           
 |   |       |       ____|______       
 NP  |       NP     |           NP    
 |   |       |      |           |      
PRP VBP      NN     IN         NNS    
 |   |       |      |           |      
 I  eat  spaghetti with     chopsticks

Dependency parse tree: 
id: 1	word: I	head id: 2	head: eat	deprel: nsubj
id: 2	word: eat	head id: 0	head: root	deprel: root
id: 3	word: spaghetti	head id: 2	head: eat	deprel: obj
id: 4	word: with	head id: 5	head: chopsticks	deprel: case
id: 5	word: chopsticks	head id: 2	head: eat	deprel: obl


In [ ]:
sent_2 = "I eat spaghetti with meatballs"

print_result(sent_2)

Sentence:  I eat spaghetti with meatballs
Constituency parse tree: 
(ROOT (S (NP (PRP I)) (VP (VBP eat) (NP (NN spaghetti)) (PP (IN with) (NP (NNS meatballs))))))
    ROOT                             
     |                                
     S                               
  ___|_______                         
 |           VP                      
 |    _______|___________             
 |   |       |           PP          
 |   |       |       ____|______      
 NP  |       NP     |           NP   
 |   |       |      |           |     
PRP VBP      NN     IN         NNS   
 |   |       |      |           |     
 I  eat  spaghetti with     meatballs

Dependency parse tree: 
id: 1	word: I	head id: 2	head: eat	deprel: nsubj
id: 2	word: eat	head id: 0	head: root	deprel: root
id: 3	word: spaghetti	head id: 2	head: eat	deprel: obj
id: 4	word: with	head id: 5	head: meatballs	deprel: case
id: 5	word: meatballs	head id: 2	head: eat	deprel: obl


# B

In [ ]:
sent_3 = "She doesn't like me with glasses"

print_result(sent_3)

Sentence:  She doesn't like me with glasses
Constituency parse tree: 
(ROOT (S (NP (PRP She)) (VP (VBZ does) (RB n't) (VP (VB like) (NP (PRP me)) (PP (IN with) (NP (NNS glasses)))))))
    ROOT                              
     |                                 
     S                                
  ___|________                         
 |            VP                      
 |    ________|____                    
 |   |    |        VP                 
 |   |    |    ____|________           
 |   |    |   |    |        PP        
 |   |    |   |    |    ____|_____     
 NP  |    |   |    NP  |          NP  
 |   |    |   |    |   |          |    
PRP VBZ   RB  VB  PRP  IN        NNS  
 |   |    |   |    |   |          |    
She does n't like  me with     glasses

Dependency parse tree: 
id: 1	word: She	head id: 4	head: like	deprel: nsubj
id: 2	word: does	head id: 4	head: like	deprel: aux
id: 3	word: n't	head id: 4	head: like	deprel: advmod
id: 4	word: like	head id: 0	head: root	depr

In [ ]:
sent_4 = "I saw the girl with my glasses"

print_result(sent_4)

Sentence:  I saw the girl with my glasses
Constituency parse tree: 
(ROOT (S (NP (PRP I)) (VP (VBD saw) (NP (DT the) (NN girl)) (PP (IN with) (NP (PRP$ my) (NNS glasses))))))
    ROOT                                   
     |                                      
     S                                     
  ___|____________                          
 |                VP                       
 |    ____________|_________                
 |   |        |             PP             
 |   |        |         ____|____           
 NP  |        NP       |         NP        
 |   |     ___|___     |     ____|_____     
PRP VBD   DT      NN   IN  PRP$       NNS  
 |   |    |       |    |    |          |    
 I  saw  the     girl with  my      glasses

Dependency parse tree: 
id: 1	word: I	head id: 2	head: saw	deprel: nsubj
id: 2	word: saw	head id: 0	head: root	deprel: root
id: 3	word: the	head id: 4	head: girl	deprel: det
id: 4	word: girl	head id: 2	head: saw	deprel: obj
id: 5	word: with	head 

# C

In [ ]:
# spacy isnt able to draw sentences with punctuation
sent_5 = "The table consists of all kinds of food, including soups, roasts, mashes, puddings, and fruits"

print_result(sent_5)

Sentence:  The table consists of all kinds of food, including soups, roasts, mashes, puddings, and fruits
Constituency parse tree: 
(ROOT (S (NP (DT The) (NN table)) (VP (VBZ consists) (PP (IN of) (NP (NP (DT all) (NNS kinds)) (PP (IN of) (NP (NN food))) (, ,) (PP (VBG including) (NP (NP (NNS soups)) (, ,) (NP (NNS roasts)) (, ,) (NP (NNS mashes)) (, ,) (NP (NNS puddings)) (, ,) (CC and) (NP (NNS fruits)))))))))
         ROOT                                                                                                              
          |                                                                                                                 
          S                                                                                                                
      ____|_____________                                                                                                    
     |                  VP                                                            

RuntimeError: [E039] Array bounds exceeded while searching for root word. This likely means the parse tree is in an invalid state. Please report this issue here: http://github.com/explosion/spaCy/issues